In [1]:
import os
import glob
import librosa
import matplotlib.pyplot as plt
import numpy as np
import sys
from tqdm import tqdm
from scipy.spatial.distance import euclidean
import pandas as pd
from fastdtw import fastdtw

In [2]:
audio_path_unlabeled = glob.glob(os.path.join('C:./dataset/audioonly/unlabeled', '*.wav'))
dataset_maindir = os.path.join(os.getcwd(), 'dataset')
sub_dir_name = next(os.walk(dataset_maindir))[1]
sub_dir_name

['.ipynb_checkpoints', 'audioonly', 'spectrograms']

In [3]:
sub_dir_path = os.path.join(dataset_maindir, sub_dir_name[1])
sub_dir_subpath = next(os.walk(sub_dir_path))[1]
sub_dir_subpath

['labeled', 'unlabeled']

In [4]:
class_folder_path = os.path.join(sub_dir_path, sub_dir_subpath[0])
class_lst = next(os.walk(class_folder_path))[1]
class_path = [os.path.join(class_folder_path, type) for type in class_lst]
class_path

['C:\\Users\\dave\\aiffel\\EUANGGG\\maincode\\data\\dataset\\audioonly\\labeled\\belly_pain',
 'C:\\Users\\dave\\aiffel\\EUANGGG\\maincode\\data\\dataset\\audioonly\\labeled\\burping',
 'C:\\Users\\dave\\aiffel\\EUANGGG\\maincode\\data\\dataset\\audioonly\\labeled\\discomfort',
 'C:\\Users\\dave\\aiffel\\EUANGGG\\maincode\\data\\dataset\\audioonly\\labeled\\hungry',
 'C:\\Users\\dave\\aiffel\\EUANGGG\\maincode\\data\\dataset\\audioonly\\labeled\\tired']

In [5]:
audio_path_labeled = [glob.glob(os.path.join(folder,'*.wav')) for folder in class_path]

In [6]:
bellypain_path = audio_path_labeled[0]
burping_path = audio_path_labeled[1]
discomfort_path = audio_path_labeled[2]
hungry_path = audio_path_labeled[3]
tired_path = audio_path_labeled[4]

In [7]:
# Bellypain audio to MFCCs
spectro_belly = []

for path in bellypain_path:
    y, sr = librosa.load(path, sr = 44100)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    spectro_belly.append(mfccs)

In [8]:
# burping audio to MFCCs
spectro_burp = []

for path in burping_path:
    y, sr = librosa.load(path, sr = 44100)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    spectro_burp.append(mfccs)

In [9]:
# discomfort audio to MFCCs
spectro_discomfort = []

for path in discomfort_path:
    y, sr = librosa.load(path, sr = 44100)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    spectro_discomfort.append(mfccs)

In [10]:
# hungry audio to MFCCs
spectro_hungry = []

for path in hungry_path:
    y, sr = librosa.load(path, sr = 44100)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    spectro_hungry.append(mfccs)

In [11]:
# tired audio to MFCCs
spectro_tired = []

for path in tired_path:
    y, sr = librosa.load(path, sr = 44100)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    spectro_tired.append(mfccs)

In [12]:
# tired audio to MFCCs
spectro_unlabeled = []

for path in tqdm(audio_path_unlabeled):
    y, sr = librosa.load(path, sr = 44100)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    spectro_unlabeled.append(mfccs)

100%|███████████████████████████████████████████████████████████████████████████| 18190/18190 [01:42<00:00, 176.90it/s]


In [13]:
labeled_mfccs = {'bellypain': spectro_belly, 
                'burp': spectro_burp,
                'discomfort': spectro_discomfort,
                'hungry': spectro_hungry,
                'tired': spectro_tired}

In [14]:
from sklearn.cluster import KMeans
representative_mfccs = {}

for label, mfccs in labeled_mfccs.items():
    all_mfccs = np.concatenate(mfccs, axis=1)
    kmeans = KMeans(n_clusters=1, random_state=0).fit(all_mfccs.T)
    representative_mfccs[label] = kmeans.cluster_centers_[0]

C:\Users\dave\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\dave\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\dave\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\dave\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\

In [16]:
# Initialize lists to store predicted classes and distances
predicted_classes = []
raw_distances = []

# Loop through each unlabeled data point
for unlabeled_mfcc in tqdm(spectro_unlabeled):
    min_distance = float('inf')
    predicted_class = None
    for label, rep_mfcc in representative_mfccs.items():
        distance, _ = fastdtw(unlabeled_mfcc.T, rep_mfcc.reshape(1, -1), dist=euclidean)
        if distance < min_distance:
            min_distance = distance
            predicted_class = label
    predicted_classes.append(predicted_class)
    raw_distances.append(min_distance)

# Normalize the distances inversely
# One simple way is to take the reciprocal of the distance
# However, to avoid division by zero, add a small constant (e.g., 1e-5)
# normalized_distances = [1 / (d + 1e-5) for d in raw_distances]

# Convert to a DataFrame
df_predictions = pd.DataFrame({
    'Predicted_Class': predicted_classes,
    'Raw_Distance': raw_distances
})

100%|███████████████████████████████████████████████████████████████████████████| 18190/18190 [01:13<00:00, 245.90it/s]


In [17]:
df_predictions

,Predicted_Class,Raw_Distance
0,tired,18762.765544
1,tired,19050.995268
2,tired,24339.702591
3,tired,36375.280190
4,tired,19315.015702
...,...,...
18185,tired,18938.816106
18186,bellypain,18351.961166
18187,bellypain,18069.502765
18188,bellypain,19143.633368


In [18]:
df_predictions['Raw_Distance'].describe()

count     18190.000000
mean      16653.571308
std       10027.565217
min        2133.268549
25%       10734.060885
50%       14406.772987
75%       19752.209099
max      256247.198482
Name: Raw_Distance, dtype: float64

In [31]:
cond = (df_predictions['Raw_Distance'] < 20000)
outlier_excluded = df_predictions[cond]

In [35]:
outlier_excluded

,Predicted_Class,Raw_Distance
0,tired,18762.765544
1,tired,19050.995268
4,tired,19315.015702
6,hungry,14441.563902
7,tired,15945.209625
...,...,...
18185,tired,18938.816106
18186,bellypain,18351.961166
18187,bellypain,18069.502765
18188,bellypain,19143.633368


In [36]:
MIN_DISTANCE = 2133
MAX_DISTANCE = 20000

normalized_dist = [(MAX_DISTANCE - d) / (MAX_DISTANCE - MIN_DISTANCE) for d in outlier_excluded['Raw_Distance']]

df_pred = pd.DataFrame({
    'Predicted_Class': outlier_excluded['Predicted_Class'],
    'Normalized_Distance': normalized_dist
})

In [37]:
df_pred

,Predicted_Class,Normalized_Distance
0,tired,0.069247
1,tired,0.053115
4,tired,0.038338
6,hungry,0.311101
7,tired,0.226943
...,...,...
18185,tired,0.059394
18186,bellypain,0.092239
18187,bellypain,0.108048
18188,bellypain,0.047930


In [38]:
df_pred.describe()

,Normalized_Distance
count,13766.000000
mean,0.421116
std,0.227752
min,0.000019
25%,0.249966
50%,0.409847
75%,0.572330
max,0.999985


In [47]:
cond = df_pred['Normalized_Distance'] > 0.90
df_pred[cond]['Predicted_Class'].value_counts()

Predicted_Class
tired        243
bellypain     40
burp           1
Name: count, dtype: int64

In [48]:
df_predictions.columns

Index(['Predicted_Class', 'Raw_Distance'], dtype='object')

In [50]:
df_predictions.rename(columns = {'Predicted_Class': 'pred_class_dtw',
                               'Raw_Distance': 'raw_dist_dtw'})

,pred_class_dtw,raw_dist_dtw
0,tired,18762.765544
1,tired,19050.995268
2,tired,24339.702591
3,tired,36375.280190
4,tired,19315.015702
...,...,...
18185,tired,18938.816106
18186,bellypain,18351.961166
18187,bellypain,18069.502765
18188,bellypain,19143.633368


In [52]:
df_predictions.to_csv(os.path.join('C:./', 'dtw_sim.csv'))